In [33]:
# ================================================================
# RESUMEN DEL FLUJO DE TRABAJO Y ARCHIVOS GENERADOS
# ================================================================
# 
# En este notebook se desarrolla el proceso desde la extracción 
# de la grilla completa a partir de archivos WRF (`wrfout`) hasta
# la generación de mapas para validación visual y un pre-análisis 
# numérico.
#
# ------------------------------------------------
# Distinción entre archivos _geo y estándar:
# ------------------------------------------------
# - Archivos con sufijo `_geo.pkl` → Guardados como GeoDataFrame
#   (contienen columna 'geometry' y CRS, para operaciones espaciales).
# - Archivos sin sufijo `_geo.pkl` → Guardados como DataFrame
#   estándar de pandas (sin columna 'geometry').
#
# ------------------------------------------------
# Bloques de trabajo:
# ------------------------------------------------
# 1. Extracción de grilla completa (D03 y D04)
#    - A partir de archivos WRF (`wrfout_d03` y `wrfout_d04`).
#    - Se obtienen coordenadas (i, j, lat, lon).
#    - Salida en formato GeoDataFrame y DataFrame.
#
# 2. Filtrado de grilla sobre superficie terrestre
#    - Usando shapefile de regiones (`regiones.json`).
#    - Se filtran puntos dentro de la Región de Valparaíso.
#    - Salida en formato GeoDataFrame y DataFrame.
#
# 3. Generación de mapa combinado para validación visual
#    - Un solo mapa con:
#        a) Contornos de dominios completos (D03 y D04).
#        b) Puntos de superficie filtrados para cada dominio.
#    - Salida: `dominios_y_superficie.html`
#
# 4. Pre-análisis numérico
#    - Cálculo de:
#        a) Número de puntos sobre superficie para cada dominio.
#        b) Área cubierta por esos puntos (km²).
#        c) Porcentaje de cobertura respecto al área total de la región.
#    - Resultados mostrados en tabla por consola.
#
# ------------------------------------------------
#  Archivos de salida:
# ------------------------------------------------
# - grilla_WRF-D03_geo.pkl                → Grilla completa D03 (GeoDataFrame)
# - grilla_WRF-D03.pkl                     → Grilla completa D03 (DataFrame)
# - grilla_WRF-D04_geo.pkl                → Grilla completa D04 (GeoDataFrame)
# - grilla_WRF-D04.pkl                     → Grilla completa D04 (DataFrame)
# - grilla_WRF-Superficie-D03_geo.pkl     → Grilla filtrada superficie D03 (GeoDataFrame)
# - grilla_WRF-Superficie-D03.pkl          → Grilla filtrada superficie D03 (DataFrame)
# - grilla_WRF-Superficie-D04_geo.pkl     → Grilla filtrada superficie D04 (GeoDataFrame)
# - grilla_WRF-Superficie-D04.pkl          → Grilla filtrada superficie D04 (DataFrame)
# - dominios_y_superficie.html            → Mapa de contornos y puntos para validación
#
# Nota: Los archivos WRF (`wrfout_...`) utilizados para generar 
# las grillas NO están disponibles en este repositorio.
# ================================================================


In [26]:
from netCDF4 import Dataset
from wrf import getvar
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

# ==============================
# Función para generar la grilla
# ==============================
def generar_grilla(wrf_path, dominio):
    # Leer archivo WRF
    wrf_file = Dataset(wrf_path)
    lons = getvar(wrf_file, "lon").values
    lats = getvar(wrf_file, "lat").values

    # Crear lista de filas con i, j, lat, lon
    rows = []
    for i in range(lats.shape[0]):
        for j in range(lats.shape[1]):
            rows.append({
                "i": i,
                "j": j,
                "lat": lats[i, j],
                "lon": lons[i, j],
                "geometry": Point(lons[i, j], lats[i, j])
            })

    # Crear GeoDataFrame
    grilla_gdf = gpd.GeoDataFrame(rows, crs="EPSG:4326")
    
    # Guardar con geometría
    gdf_path = f"grilla_WRF-{dominio}_geo.pkl"
    grilla_gdf.to_pickle(gdf_path)
    print(f"[OK] Guardado GeoDataFrame: {gdf_path}")

    # Guardar sin geometría
    df_path = f"grilla_WRF-{dominio}.pkl"
    grilla_gdf.drop(columns="geometry").to_pickle(df_path)
    print(f"[OK] Guardado DataFrame: {df_path}")

    return grilla_gdf

# ==============================
# Generar para ambos dominios
# ==============================
wrf_d03 = "wrfout_d03_2015-05-28_00:00:00"
wrf_d04 = "wrfout_d04_2018-05-27_00:00:00"

grilla_d03 = generar_grilla(wrf_d03, "D03")
grilla_d04 = generar_grilla(wrf_d04, "D04")


[OK] Guardado GeoDataFrame: grilla_WRF-D03_geo.pkl
[OK] Guardado DataFrame: grilla_WRF-D03.pkl
[OK] Guardado GeoDataFrame: grilla_WRF-D04_geo.pkl
[OK] Guardado DataFrame: grilla_WRF-D04.pkl


In [27]:
import geopandas as gpd
import pandas as pd

def filtrar_superficie(pkl_path, dominio, region_path="regiones.json"):
    # Cargar grilla
    grilla_df = pd.read_pickle(pkl_path)
    
    # Convertir a GeoDataFrame si es DataFrame normal
    if not isinstance(grilla_df, gpd.GeoDataFrame):
        grilla_df = gpd.GeoDataFrame(
            grilla_df,
            geometry=gpd.points_from_xy(grilla_df["lon"], grilla_df["lat"]),
            crs="EPSG:4326"
        )

    # Cargar regiones y filtrar Valparaíso
    regiones_gdf = gpd.read_file(region_path).to_crs("EPSG:4326")
    valpo_region = regiones_gdf[regiones_gdf["Region"] == "Región de Valparaíso"]

    # Unir geometrías en un solo polígono (uso de union_all en lugar de unary_union)
    poligono_tierra = valpo_region.union_all()

    # Filtrar puntos dentro de la región
    grilla_superficie = grilla_df[grilla_df.geometry.within(poligono_tierra)]

    print(f"[{dominio}] Puntos originales: {len(grilla_df)}")
    print(f"[{dominio}] Puntos sobre superficie: {len(grilla_superficie)}")

    # Guardar versión GeoDataFrame
    out_geo = f"grilla_WRF-Superficie-{dominio}_geo.pkl"
    grilla_superficie.to_pickle(out_geo)
    print(f"[OK] Guardado GeoDataFrame: {out_geo}")

    # Guardar versión DataFrame sin geometría
    out_df = f"grilla_WRF-Superficie-{dominio}.pkl"
    grilla_superficie.drop(columns="geometry").to_pickle(out_df)
    print(f"[OK] Guardado DataFrame: {out_df}")

    return grilla_superficie

# Filtrar para ambos dominios
filtrar_superficie("grilla_WRF-D03.pkl", "D03")
filtrar_superficie("grilla_WRF-D04.pkl", "D04")


[D03] Puntos originales: 1521
[D03] Puntos sobre superficie: 474
[OK] Guardado GeoDataFrame: grilla_WRF-Superficie-D03_geo.pkl
[OK] Guardado DataFrame: grilla_WRF-Superficie-D03.pkl
[D04] Puntos originales: 1521
[D04] Puntos sobre superficie: 773
[OK] Guardado GeoDataFrame: grilla_WRF-Superficie-D04_geo.pkl
[OK] Guardado DataFrame: grilla_WRF-Superficie-D04.pkl


,i,j,lat,lon,geometry
11,0,11,-33.178204,-71.683090,POINT (-71.68309 -33.1782)
12,0,12,-33.178204,-71.672699,POINT (-71.6727 -33.1782)
13,0,13,-33.178204,-71.662315,POINT (-71.66232 -33.1782)
14,0,14,-33.178204,-71.651932,POINT (-71.65193 -33.1782)
15,0,15,-33.178204,-71.641541,POINT (-71.64154 -33.1782)
...,...,...,...,...,...
1516,38,34,-32.847248,-71.444206,POINT (-71.44421 -32.84725)
1517,38,35,-32.847248,-71.433823,POINT (-71.43382 -32.84725)
1518,38,36,-32.847248,-71.423431,POINT (-71.42343 -32.84725)
1519,38,37,-32.847248,-71.413048,POINT (-71.41305 -32.84725)


In [28]:
import folium
import pandas as pd
import geopandas as gpd

# Función para cargar GeoDataFrame desde PKL
def cargar_geo(pkl_geo):
    gdf = pd.read_pickle(pkl_geo)
    if not isinstance(gdf, gpd.GeoDataFrame):
        gdf = gpd.GeoDataFrame(
            gdf,
            geometry=gpd.points_from_xy(gdf["lon"], gdf["lat"]),
            crs="EPSG:4326"
        )
    return gdf

# Cargar grillas de superficie
gdf_d03 = cargar_geo("grilla_WRF-Superficie-D03_geo.pkl")
gdf_d04 = cargar_geo("grilla_WRF-Superficie-D04_geo.pkl")

# Crear mapa centrado en la media de coordenadas combinadas
centro = [
    (gdf_d03["lat"].mean() + gdf_d04["lat"].mean()) / 2,
    (gdf_d03["lon"].mean() + gdf_d04["lon"].mean()) / 2
]
m = folium.Map(location=centro, zoom_start=8, tiles="CartoDB positron")

# Calcular contornos usando union_all() en lugar de unary_union
contorno_d03 = gdf_d03.union_all().convex_hull
contorno_d04 = gdf_d04.union_all().convex_hull

# Añadir contorno D03 (azul)
folium.GeoJson(
    contorno_d03,
    name="Contorno D03",
    style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0}
).add_to(m)

# Añadir contorno D04 (verde)
folium.GeoJson(
    contorno_d04,
    name="Contorno D04",
    style_function=lambda x: {"color": "green", "weight": 2, "fillOpacity": 0}
).add_to(m)

# Añadir capa de control
folium.LayerControl().add_to(m)

# Guardar mapa
m.save("contorno_D03_D04_contrastado.html")
print("[OK] Mapa contrastado guardado como contorno_D03_D04_contrastado.html")


[OK] Mapa contrastado guardado como contorno_D03_D04_contrastado.html


In [24]:
m

In [31]:
import folium
import pandas as pd
import geopandas as gpd

def cargar_geo(pkl_path):
    gdf = pd.read_pickle(pkl_path)
    if not isinstance(gdf, gpd.GeoDataFrame):
        gdf = gpd.GeoDataFrame(
            gdf,
            geometry=gpd.points_from_xy(gdf["lon"], gdf["lat"]),
            crs="EPSG:4326"
        )
    return gdf

# === Cargar datos ===
# Dominios completos
gdf_d03_full = cargar_geo("grilla_WRF-D03_geo.pkl")
gdf_d04_full = cargar_geo("grilla_WRF-D04_geo.pkl")

# Dominios filtrados por superficie
gdf_d03_sup = cargar_geo("grilla_WRF-Superficie-D03_geo.pkl")
gdf_d04_sup = cargar_geo("grilla_WRF-Superficie-D04_geo.pkl")

# === Crear mapa centrado ===
centro = [
    (gdf_d03_full["lat"].mean() + gdf_d04_full["lat"].mean()) / 2,
    (gdf_d03_full["lon"].mean() + gdf_d04_full["lon"].mean()) / 2
]
m = folium.Map(location=centro, zoom_start=8, tiles="CartoDB positron")

# === Añadir contornos de dominios completos ===
contorno_d03 = gdf_d03_full.union_all().convex_hull
contorno_d04 = gdf_d04_full.union_all().convex_hull

folium.GeoJson(
    contorno_d03,
    name="Contorno D03 (completo)",
    style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0}
).add_to(m)

folium.GeoJson(
    contorno_d04,
    name="Contorno D04 (completo)",
    style_function=lambda x: {"color": "green", "weight": 2, "fillOpacity": 0}
).add_to(m)

# === Añadir puntos de superficie ===
for _, row in gdf_d03_sup.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=1,
        color="blue",
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

for _, row in gdf_d04_sup.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=1,
        color="green",
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# === Añadir control de capas ===
folium.LayerControl().add_to(m)

# === Guardar mapa ===
m.save("dominios_y_superficie.html")
print("[OK] Mapa guardado como dominios_y_superficie.html")


[OK] Mapa guardado como dominios_y_superficie.html


In [32]:
m

In [25]:
import pandas as pd
import geopandas as gpd

def cargar_geo(pkl_path):
    gdf = pd.read_pickle(pkl_path)
    if not isinstance(gdf, gpd.GeoDataFrame):
        gdf = gpd.GeoDataFrame(
            gdf,
            geometry=gpd.points_from_xy(gdf["lon"], gdf["lat"]),
            crs="EPSG:4326"
        )
    return gdf

# === Cargar grillas filtradas ===
gdf_d03_sup = cargar_geo("grilla_WRF-Superficie-D03_geo.pkl")
gdf_d04_sup = cargar_geo("grilla_WRF-Superficie-D04_geo.pkl")

# === Cargar Región de Valparaíso y calcular área total ===
regiones_gdf = gpd.read_file("regiones.json").to_crs(epsg=32719)
valpo_region = regiones_gdf[regiones_gdf["Region"] == "Región de Valparaíso"]
area_region_km2 = valpo_region.union_all().area / 1e6  # km²

# === Función para calcular puntos y área cubierta ===
def calcular_area(gdf):
    gdf_proj = gdf.to_crs(epsg=32719)
    poligono = gdf_proj.union_all().convex_hull
    area_km2 = poligono.area / 1e6
    return len(gdf), area_km2

# === Calcular para cada dominio ===
resumen_data = []
for dominio, gdf in [("D03", gdf_d03_sup), ("D04", gdf_d04_sup)]:
    puntos, area_km2 = calcular_area(gdf)
    porcentaje = (area_km2 / area_region_km2) * 100
    resumen_data.append({
        "Dominio": dominio,
        "Puntos en superficie": puntos,
        "Área cubierta (km²)": round(area_km2, 2),
        "Cobertura respecto a la región (%)": round(porcentaje, 2)
    })

# === Crear y mostrar tabla ===
resumen_df = pd.DataFrame(resumen_data)
print(resumen_df.to_string(index=False, justify="center"))


Dominio  Puntos en superficie  Área cubierta (km²)  Cobertura respecto a la región (%)
  D03            474                4232.25                       25.95               
  D04            773                 828.13                        5.08               
